In [1]:
# Import required libraries
# Add whatever you want
import tensorflow as tf
import numpy as np
import os
import scipy.io as spio
import matplotlib.pyplot as plt
%matplotlib inline

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Test tensorflow
print('TensorFlow version: ' + tf.__version__)
a = tf.constant(2.0)
b = tf.constant(3.0)
c = a * b

sess = tf.Session()
result = sess.run([a, b, c])
print('%f * %f = %f' % (result[0], result[1], result[2]))
sess.close()

# load data
TRAIN_FOLDER = '../data/normalized/train'
TEST_FOLDER = '../data/normalized/test'
ID_FILE = '../data/normalized/id.json'
NUM_SPEAKERS = 200

def get_attributes(fname):
    attr = fname.split('.')[0].split('-')
    dialect = attr[0]
    gender = attr[1][0]
    speaker_id = attr[1]
    sentence_type = attr[2][:2]
    return dialect, gender, speaker_id, sentence_type

def load_files(train):
    X = []
    Y = []
    for speaker_id, flist in train.items():
        for fname in flist:
            filedata = spio.loadmat(fname)['data'][0, :]
            for segment in filedata:
                for x in segment:
                    X.append(x) # dim(x) => (390,)
                    Y.append(speaker_id)
    return X, Y
    

def load_train_data():
    import json
    with open(ID_FILE, 'r') as f_json:
        ids = json.load(f_json)
    print('ids', ids)
    
    
    files = [f for f in os.listdir(TRAIN_FOLDER) if os.path.isfile(os.path.join(TRAIN_FOLDER, f))]

    # split train val data
    train = {}
    val = {}
    
    added = {}
    for file in files:
        dialect, gender, speaker_id, sentence_type = get_attributes(file)
        file_path = os.path.join(TRAIN_FOLDER, file)
        if speaker_id not in ids:
            print('ERROR:', speaker_id, 'not found in ids')
        speaker_id = ids[speaker_id]
        val_set = added.setdefault(speaker_id, {})
        if sentence_type not in val_set:
            val.setdefault(speaker_id, []).append(file_path)
            val_set[sentence_type] = True
        else:
            train.setdefault(speaker_id, []).append(file_path)
    # test data    
    test = {}
    files = [f for f in os.listdir(TEST_FOLDER) if os.path.isfile(os.path.join(TEST_FOLDER, f))]
    for file in files:
        dialect, gender, speaker_id, sentence_type = get_attributes(file)
        file_path = os.path.join(TEST_FOLDER, file)
        if speaker_id not in ids:
            print('ERROR:', speaker_id, 'not found in ids')
        speaker_id = ids[speaker_id]
        test.setdefault(speaker_id, []).append(file_path)
    
    # load data
    # input_path = os.path.join(TRAIN_FOLDER, file)
    X_train, Y_train = load_files(train)
    X_val, Y_val = load_files(val)
    X_test, Y_test = load_files(test)
    return np.array(X_train), np.array(Y_train), np.array(X_val), np.array(Y_val), np.array(X_test), np.array(Y_test)
    
print('Loading data...')
X_train, Y_train, X_val, Y_val, X_test, Y_test = load_train_data()

print()
print('# Train :', len(Y_train))
print('# Eval  :', len(Y_val))
print('# Test  :', len(Y_test))
print()

# model
def fully_connected(input, num_outputs, act_fn=None):
    return tf.contrib.layers.fully_connected(input, num_outputs,
                                             activation_fn=act_fn,
                                             weights_initializer=tf.random_normal_initializer(0.0, 0.02),
                                             weights_regularizer=tf.contrib.layers.l2_regularizer(scale=0.0001)
                                            )

class BaseModel(object):
    def __init__(self):
        self.batch_size = 128
        self.log_step = 50
        self._build_model()

    def _model(self):
        print('-' * 5 + '  Sample model  ' + '-' * 5)

        print('input layer: ' + str(self.X.get_shape()))

        with tf.variable_scope('fc1'):
            
            self.fc1 = fully_connected(self.X, 390)
            self.relu1 = tf.nn.relu(self.fc1)
            
            print('fc1 layer: ' + str(self.relu1.get_shape()))

        with tf.variable_scope('fc2'):
            
            self.fc2 = fully_connected(self.relu1, 200)
            self.relu2 = tf.nn.relu(self.fc2)
            
            print('fc2 layer: ' + str(self.relu2.get_shape()))
            
        with tf.variable_scope('fc3'):
            
            self.fc3 = fully_connected(self.relu2, 200)
            
            print('fc3 layer: ' + str(self.fc3.get_shape()))
        
        # Return the last layer
        return self.fc3

    def _input_ops(self):
        # Placeholders
        self.X = tf.placeholder(tf.float32, [None, 390])
        self.Y = tf.placeholder(tf.int64, [None])
        
        self.global_step = tf.Variable(0, trainable=False)
        self.starter_learning_rate = 4e-4
        
        # tf.train.exponential_decay(learning_rate, global_step, decay_steps, 
        # decay_rate, staircase=False, name=None)
        # staircase: Boolean. If True decay the learning rate at discrete intervals
        
        # decay every 500 steps with a base of 0.96
        self.learning_rate = tf.train.exponential_decay(self.starter_learning_rate,
                                                        self.global_step, 500, 0.96,
                                                        staircase=True)
        

    def _build_optimizer(self):
        # Adam optimizer 'self.train_op' that minimizes 'self.loss_op'
        
        # Passing global_step to minimize() will increment it at each step.
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.train_op = optimizer.minimize(self.loss_op, global_step=self.global_step)
        
        
    def _loss(self, labels, logits):
        # Softmax cross entropy loss 'self.loss_op'
        
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
        self.loss_op = tf.reduce_mean(loss)     
        

    def _build_model(self):
        # Define input variables
        self._input_ops()

        # Convert Y to one-hot vector
        labels = tf.one_hot(self.Y, NUM_SPEAKERS)

        # Build a model and get logits
        logits = self._model()

        # Compute loss
        self._loss(labels, logits)
        
        # Build optimizer
        self._build_optimizer()

        # Compute accuracy
        predict = tf.argmax(logits, 1)
        correct = tf.equal(predict, self.Y)
        self.accuracy_op = tf.reduce_mean(tf.cast(correct, tf.float32))
        
    def train(self, sess, X_train, Y_train, X_val, Y_val, num_epoch=40):
        # shuffle
        from sklearn.utils import shuffle
        X_train, Y_train = shuffle(X_train, Y_train)
        
        # try loading checkpoint
        saver = tf.train.Saver()
        save_dir = "tf_models/"        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        save_path = os.path.join(save_dir, 'csci-599_proj')
        
        try:
            print("Trying to restore last checkpoint ...")

            # Use TensorFlow to find the latest checkpoint - if any.
            last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=save_dir)

            # Try and load the data in the checkpoint.
            saver.restore(sess, save_path=last_chk_path)

            # If we get to this point, the checkpoint was successfully loaded.
            print("Restored checkpoint from:", last_chk_path)
        except:
            # If the above failed for some reason, simply
            # initialize all the variables for the TensorFlow graph.
            print("Failed to restore checkpoint. Initializing variables instead.")
            sess.run(tf.global_variables_initializer())

        step = 0
        losses = []
        accuracies = []
        num_training = len(Y_train)
        print('-' * 5 + '  Start training  ' + '-' * 5)
        for epoch in range(num_epoch):
            print('train for epoch %d' % epoch)
            print('global step: %d' % self.global_step.eval())
            for i in range(num_training // self.batch_size):
                X_ = X_train[i * self.batch_size:(i + 1) * self.batch_size][:]
                Y_ = Y_train[i * self.batch_size:(i + 1) * self.batch_size]
                
                feed_dict = {
                    self.X: X_,
                    self.Y: Y_
                }                
                
                fetches = [self.train_op, self.loss_op, self.accuracy_op]

                _, loss, accuracy = sess.run(fetches, feed_dict=feed_dict)
                losses.append(loss)
                accuracies.append(accuracy)

                if step % self.log_step == 0:
                    print('iteration (%d): loss = %.3f, accuracy = %.3f' %
                        (step, loss, accuracy))
                step += 1
            
            # save state
            model_path = saver.save(sess,
                       save_path=save_path,
                       global_step=self.global_step)
            print("Model saved in %s" % model_path)

            #############################################################################
            # TODO: Plot training curves                                                #
            #############################################################################
            # Graph 1. X: epoch, Y: training loss
            plt.plot(losses)
            plt.title('training loss')
            plt.xlabel('iteration')
            plt.ylabel('loss')
            plt.show()    

            # Graph 2. X: epoch, Y: training accuracy
            plt.plot(accuracies)
            plt.title('training accuracy')
            plt.xlabel('iteration')
            plt.ylabel('accuracy')
            plt.show()

            # Print validation results
            print('validation for epoch %d' % epoch)
            val_accuracy = self.evaluate(sess, X_val, Y_val)
            print('-  epoch %d: validation accuracy = %.3f' % (epoch, val_accuracy))

    def evaluate(self, sess, X_eval, Y_eval):
        eval_accuracy = 0.0
        eval_iter = 0
        for i in range(X_eval.shape[0] // self.batch_size):
            X_ = X_eval[i * self.batch_size:(i + 1) * self.batch_size][:]
            Y_ = Y_eval[i * self.batch_size:(i + 1) * self.batch_size]
                        
            
            feed_dict = {
                    self.X: X_,
                    self.Y: Y_
                }
            
            accuracy = sess.run(self.accuracy_op, feed_dict=feed_dict)
            eval_accuracy += accuracy
            eval_iter += 1
        return eval_accuracy / eval_iter


TensorFlow version: 1.3.1
2.000000 * 3.000000 = 6.000000
Loading data...
ids {'MCSS0': 21, 'MDLC2': 36, 'MMSM0': 129, 'MDSS0': 44, 'MTAS0': 177, 'MDBP0': 26, 'MKAM0': 103, 'MDEF0': 29, 'MRTJ0': 167, 'MMJB1': 127, 'MGSH0': 65, 'MAKR0': 3, 'MRCW0': 146, 'MTPP0': 188, 'MMRP0': 128, 'MMAG0': 117, 'MBJV0': 9, 'MJWT0': 99, 'MJEB0': 79, 'MSRG0': 175, 'MRJH0': 157, 'MDPK0': 41, 'MMXS0': 130, 'MTRC0': 190, 'MTQC0': 189, 'MRDS0': 148, 'MTJG0': 181, 'MEFG0': 51, 'MESG0': 53, 'MADC0': 0, 'MPGH0': 133, 'MPRD0': 137, 'MCLM0': 19, 'MPGR0': 134, 'MGXP0': 66, 'MJXL0': 100, 'MJRP0': 96, 'MDAS0': 24, 'MRSO0': 164, 'MLSH0': 115, 'MJJJ0': 84, 'MCEW0': 17, 'MJDC0': 77, 'MWSB0': 198, 'MBGT0': 8, 'MVJH0': 193, 'MKDT0': 104, 'MDLC0': 35, 'MTJM0': 182, 'MJEB1': 80, 'MDMA0': 38, 'MJMA0': 89, 'MDMT0': 39, 'MRSP0': 165, 'MMGG0': 125, 'MRAB0': 141, 'MLNS0': 114, 'MSMS0': 174, 'MRJB1': 156, 'MJPM0': 92, 'MAPV0': 4, 'MCDD0': 14, 'MRLJ0': 161, 'MCTM0': 22, 'MSMC0': 173, 'MTKP0': 184, 'MJMM0': 91, 'MRAB1': 142, 'MTPF0'

In [ ]:
import os
import time
from datetime import datetime

start_time = time.time()
print(datetime.now())
print()

# Clear old computation graphs
tf.reset_default_graph()

# Train our sample model
with tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True)) as sess:
    with tf.device('/gpu:0'):
        model = BaseModel()
        model.train(sess, X_train, Y_train, X_val, Y_val, num_epoch=40)
        accuracy = model.evaluate(sess, X_test, Y_test)
        print('***** test accuracy: %.3f' % accuracy)
        
print('Done')
print(datetime.now())
print("Total time: %s seconds" % (time.time() - start_time))